In [1]:
import warnings


warnings.filterwarnings("ignore")

import nltk
from nltk import FreqDist
nltk.download('stopwords')

# !pip install pyLDAvis

import pandas as pd
# pd.set_option("display.max_colwidth", 200)
import numpy as np
import json
import re
import gzip
import spacy

import gensim
from gensim import corpora

import pyLDAvis
# import pyLDAvis.gensim
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from gensim.models.coherencemodel import CoherenceModel

from nltk.corpus import stopwords

import nltk

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer

from nltk.stem import WordNetLemmatizer

from tqdm import tqdm

import gensim
from gensim import corpora

import pyLDAvis
# import pyLDAvis.gensim
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tanma\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


C:\Users\tanma\Anaconda3\lib\site-packages\matplotlib\colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sized


In [2]:
import json

In [3]:
from gensim.test.utils import datapath

## Inspecting the best review model

In [20]:
engaged_df = pd.read_csv("./data/engaged_df.csv")

In [21]:
engaged_df.head()

,book_id,user_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments,clean_reviews
0,5805,4cbecbc15af3db041a8e0f594c642bb5,58f2301bd2d4bbfc1b51e4e5fb161cfe,5,"Remember, remember, the fifth of November. Thi...",Wed Jun 13 17:55:53 -0700 2012,Mon Jan 30 05:58:01 -0800 2017,Thu Jun 14 00:00:00 -0700 2012,NaN,7,0,"['remember', 'remember', 'fifth', 'part', 'quo..."
1,5805,49cc59f1c479d698507627b401d47ecf,761a17f52538341a085b629a316204a1,4,Tinha apontado este livro como um dos que tinh...,Sun Jan 05 03:27:10 -0800 2014,Tue Oct 28 15:58:20 -0700 2014,Tue Oct 28 15:58:20 -0700 2014,NaN,1,0,"['tinha', 'visto', 'portman', 'hora', 'livro',..."
2,5805,5f03864c758bfceb6d7d5e93eeb20044,ac54c3ce0c9f660c03881b0668f79c60,5,Review coming soon! www.youtube.com/ReadTomes,Tue Feb 12 11:38:36 -0800 2013,Tue Feb 12 12:00:47 -0800 2013,Tue Feb 12 12:00:47 -0800 2013,Tue Feb 12 00:00:00 -0800 2013,1,0,"['review', 'ing', 'soon', 'readtome']"
3,5805,c309dff1695ed8558b29ea8dcd7479b8,0da0bcc469c2acd15350f9a8f0a74e2b,5,"What better way to celebrate Guy Fawkes Day, t...",Tue May 20 09:52:36 -0700 2014,Thu Mar 16 09:49:11 -0700 2017,Wed Nov 05 14:22:10 -0800 2014,Wed Nov 05 00:00:00 -0800 2014,2,0,"['well', 'way', 'celebrate', 'guy', 'day', 're..."
4,5805,0f6b8c04f811e05c8978bd6b66ce7685,7d670b6c8cac0c086e21ae2f1af6eccb,4,To note - I am writing this review a quarter c...,Tue Jul 03 16:43:26 -0700 2012,Fri Feb 27 14:46:00 -0800 2015,Tue Jan 01 00:00:00 -0800 1991,NaN,10,0,"['note', 'write', 'review', 'quarter', 'centur..."


In [41]:
def to_list(x):
    x = x.strip('][').split(', ')
#     print(x)
    x = [i.strip("'") for i in x]
    return x

In [44]:
engaged_df['clean_reviews'] = engaged_df['clean_reviews'].apply(lambda x: to_list(x))

In [50]:
dictionary = corpora.Dictionary(engaged_df['clean_reviews'])

In [51]:
doc_term_matrix = tqdm([dictionary.doc2bow(rev) for rev in engaged_df['clean_reviews']])


  0%|                                                                                        | 0/39787 [01:19<?, ?it/s]


In [ ]:
lda_review_model = gensim.models.ldamodel.LdaModel.load(datapath("./data/best_review_model"))

In [48]:
lda_review_model.print_topics()

[(0,
  '0.027*"book" + 0.011*"read" + 0.010*"life" + 0.009*"story" + 0.008*"make" + 0.007*"one" + 0.006*"would" + 0.006*"like" + 0.006*"time" + 0.005*"school"'),
 (1,
  '0.140*"novel" + 0.133*"graphic" + 0.011*"con" + 0.008*"deadpool" + 0.008*"die" + 0.008*"yang" + 0.007*"kai" + 0.006*"para" + 0.006*"che" + 0.006*"manga"'),
 (2,
  '0.012*"get" + 0.009*"like" + 0.007*"volume" + 0.006*"one" + 0.006*"character" + 0.006*"man" + 0.005*"girl" + 0.005*"make" + 0.005*"thing" + 0.005*"see"'),
 (3,
  '0.024*"story" + 0.014*"batman" + 0.009*"one" + 0.008*"character" + 0.007*"good" + 0.007*"book" + 0.006*"make" + 0.006*"dark" + 0.006*"work" + 0.006*"well"'),
 (4,
  '0.030*"read" + 0.022*"like" + 0.022*"story" + 0.021*"really" + 0.020*"love" + 0.017*"book" + 0.016*"character" + 0.015*"volume" + 0.015*"one" + 0.014*"get"')]

In [78]:
pyLDAvis.enable_notebook()
# vis = pyLDAvis.gensim.prepare(lda_model, doc_term_matrix, dictionary)
vis = gensimvis.prepare(lda_review_model, doc_term_matrix, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.144846  0.038439       1        1  31.165477
2     -0.100370 -0.194305       2        1  23.060921
3     -0.107040  0.035566       3        1  21.395369
0     -0.112428  0.114412       4        1  20.961306
1      0.464684  0.005887       5        1   3.416927, topic_info=         Term          Freq         Total Category  logprob  loglift
159     novel  10275.000000  10275.000000  Default  30.0000  30.0000
93    graphic   9752.000000   9752.000000  Default  29.0000  29.0000
192      read  26049.000000  26049.000000  Default  28.0000  28.0000
386      book  26571.000000  26571.000000  Default  27.0000  27.0000
231     story  29856.000000  29856.000000  Default  26.0000  26.0000
...       ...           ...           ...      ...      ...      ...
1179      ten    287.853642    549.440388   Topic5  -5.5420   2.7300
4449      ton    273.040694    528.105833   Topic5  -5.5949   2.7168
586       non    348.389105    949.654639   Topic5  -5.3512   2.3737
929       son    292.586862   1081.783868   Topic5  -5.5257   2.0688
4216    manga    411.263470   3677.750803   Topic5  -5.1853   1.1856

[381 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
3456      1  0.995855         2nd
6094      1  0.988570         3rd
1948      1  0.980466  absolutely
1948      2  0.018722  absolutely
3435      4  0.996016     account
...     ...       ...         ...
968       3  0.295604        year
968       4  0.402807        year
1961      2  0.340373       young
1961      4  0.659537       young
5757      2  0.999202      zombie

[570 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 3, 4, 1, 2])

## Inspecting the best Description model

In [10]:
book_df = pd.read_csv('./data/book_p.csv')

In [48]:
book_df['processed_description'] = book_df['processed_description'].apply(lambda x: to_list(x))

In [49]:
book_df.head()

,isbn,text_reviews_count,series,country_code,language_code,popular_shelves,asin,is_ebook,average_rating,kindle_asin,...,edition_information,publication_year,url,image_url,book_id,ratings_count,work_id,title,title_without_series,processed_description
0,1401207928,3400,['631559'],US,eng,"[{'count': '42534', 'name': 'to-read'}, {'coun...",NaN,False,4.25,B0064W65UM,...,NaN,2005.0,https://www.goodreads.com/book/show/5805.V_for...,https://images.gr-assets.com/books/1343668985m...,5805,212464,392838,V for Vendetta,V for Vendetta,"[remember, remember, fifth, frightening, power..."
1,394541553,4402,['186595'],US,eng,"[{'count': '92915', 'name': 'to-read'}, {'coun...",NaN,False,4.35,NaN,...,NaN,1991.0,https://www.goodreads.com/book/show/15196.Maus_I,https://images.gr-assets.com/books/1327884972m...,15196,189288,1947012,Maus I: A Survivor's Tale: My Father Bleeds Hi...,Maus I: A Survivor's Tale: My Father Bleeds Hi...,"[story, jewish, survivor, son, cartoonist, try..."
2,1401207529,1790,"['482441', '300099', '194163', '636672']",US,eng,"[{'count': '7924', 'name': 'to-read'}, {'count...",NaN,False,4.22,B0064W65SO,...,NaN,2005.0,https://www.goodreads.com/book/show/59980.Batman,https://images.gr-assets.com/books/1327940389m...,59980,154013,2501570,Batman: Year One,Batman: Year One,"[lieutenant, take, new, post, crime, ride, cor..."
3,1421501686,2399,['205782'],US,eng,"[{'count': '9727', 'name': 'to-read'}, {'count...",NaN,False,4.42,NaN,...,NaN,2005.0,https://www.goodreads.com/book/show/13615.Deat...,https://images.gr-assets.com/books/1419952134m...,13615,142755,1782155,"Death Note, Vol. 1: Boredom (Death Note, #1)","Death Note, Vol. 1: Boredom (Death Note, #1)","[light, ace, student, great, prospect, bore, m..."
4,1607066017,8700,['736247'],US,eng,"[{'count': '78762', 'name': 'to-read'}, {'coun...",NaN,False,4.24,B015XEABR4,...,NaN,2012.0,https://www.goodreads.com/book/show/15704307-s...,https://images.gr-assets.com/books/1486028947m...,15704307,142640,19113524,"Saga, Vol. 1 (Saga, #1)","Saga, Vol. 1 (Saga, #1)","[two, soldier, opposite, side, never, end, gal..."


In [45]:
dictionary = corpora.Dictionary(book_df['processed_description'])

In [46]:
doc_term_matrix = tqdm([dictionary.doc2bow(rev) for rev in book_df['processed_description']])

  0%|                                                                                        | 0/39787 [00:00<?, ?it/s]

In [ ]:
lda_des_model = gensim.models.ldamodel.LdaModel.load(datapath("./data/best_description_model"))

In [48]:
lda_des_model.print_topics()

[(0,
  '0.027*"book" + 0.011*"read" + 0.010*"life" + 0.009*"story" + 0.008*"make" + 0.007*"one" + 0.006*"would" + 0.006*"like" + 0.006*"time" + 0.005*"school"'),
 (1,
  '0.140*"novel" + 0.133*"graphic" + 0.011*"con" + 0.008*"deadpool" + 0.008*"die" + 0.008*"yang" + 0.007*"kai" + 0.006*"para" + 0.006*"che" + 0.006*"manga"'),
 (2,
  '0.012*"get" + 0.009*"like" + 0.007*"volume" + 0.006*"one" + 0.006*"character" + 0.006*"man" + 0.005*"girl" + 0.005*"make" + 0.005*"thing" + 0.005*"see"'),
 (3,
  '0.024*"story" + 0.014*"batman" + 0.009*"one" + 0.008*"character" + 0.007*"good" + 0.007*"book" + 0.006*"make" + 0.006*"dark" + 0.006*"work" + 0.006*"well"'),
 (4,
  '0.030*"read" + 0.022*"like" + 0.022*"story" + 0.021*"really" + 0.020*"love" + 0.017*"book" + 0.016*"character" + 0.015*"volume" + 0.015*"one" + 0.014*"get"')]

In [78]:
pyLDAvis.enable_notebook()
# vis = pyLDAvis.gensim.prepare(lda_model, doc_term_matrix, dictionary)
vis = gensimvis.prepare(lda_des_model, doc_term_matrix, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.144846  0.038439       1        1  31.165477
2     -0.100370 -0.194305       2        1  23.060921
3     -0.107040  0.035566       3        1  21.395369
0     -0.112428  0.114412       4        1  20.961306
1      0.464684  0.005887       5        1   3.416927, topic_info=         Term          Freq         Total Category  logprob  loglift
159     novel  10275.000000  10275.000000  Default  30.0000  30.0000
93    graphic   9752.000000   9752.000000  Default  29.0000  29.0000
192      read  26049.000000  26049.000000  Default  28.0000  28.0000
386      book  26571.000000  26571.000000  Default  27.0000  27.0000
231     story  29856.000000  29856.000000  Default  26.0000  26.0000
...       ...           ...           ...      ...      ...      ...
1179      ten    287.853642    549.440388   Topic5  -5.5420   2.7300
4449      ton    273.040694    528.105833   Topic5  -5.5949   2.7168
586       non    348.389105    949.654639   Topic5  -5.3512   2.3737
929       son    292.586862   1081.783868   Topic5  -5.5257   2.0688
4216    manga    411.263470   3677.750803   Topic5  -5.1853   1.1856

[381 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
3456      1  0.995855         2nd
6094      1  0.988570         3rd
1948      1  0.980466  absolutely
1948      2  0.018722  absolutely
3435      4  0.996016     account
...     ...       ...         ...
968       3  0.295604        year
968       4  0.402807        year
1961      2  0.340373       young
1961      4  0.659537       young
5757      2  0.999202      zombie

[570 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 3, 4, 1, 2])

## Assign topics to books

In [50]:
def assign_topics_to_books(book_df, r_df, lda_model_review):
    b_df = book_df.copy()
    n_topics = len(lda_model.print_topics())
    topic_cols = []
    for i in range(n_topics):
        topic_cols.append(f'topic_{i}_review')
    print("got num topics")
    
    #based on reviews
    books = b_df['book_id'].unique()
    b_df[topic_cols] = None
    b_df['dom_topic_reviews'] = None
    for book in tqdm(books):
        test_bow = []
        for item in r_df[r_df['book_id'] == book].clean_reviews.values:
            test_bow  = test_bow + item
            
        topics = lda_model.get_document_topics(dictionary.doc2bow(test_bow))
        
        max_topic = 0
        max_prob = 0
        for topic,prob in topics:
            b_df.loc[b_df['book_id'] == book, f'topic_{topic}_review'] = prob
            if prob > max_prob:
                max_topic = topic
                max_prob = prob
        b_df.loc[b_df['book_id'] == book, 'dom_topic_reviews'] = max_topic
    print(topics)
    
    ## Based on descriptions
    
    
    b_df.fillna(0, inplace = True)
    return b_df

In [ ]:
book_df_t = assign_topics_to_books(book_df, engaged_df, lda_review_model)

In [ ]:
book_df_t.head()

In [ ]:
book_df_t.to_csv('./data/books_with_topics.csv', index = False)